# Part 1 - Question Answering

For the first part, use the Hugging Face question-answering pipeline and feed it with the five 300-word long sections from the book of your choice that you analyzed in Project 1.

These sections should be selected so they are: introducing the protagonist(s), the antagonist, the crime and crime scene, any significant evidence, and the resolution of the crime/a narrative that presents the case against the perpetrator.

For a prompt, Implement a simple prompt interface that takes in your question, runs it against the model, and returns the answer. You don't need to do anything special about this, just a simple console I/O interface without any complicated error handling. It is up to you how you want to upload the context to the model (pre-loaded into your program, on-demand, etc.).

The questions you should ask are about the identity and characteristics of the protagonist, antagonist/perpetrator, the nature and the setting of the crime or crime scene, the evidence, and the case against the perpetrator.

Document the questions, ask the questions, and document the specificity and accuracy of the results.

Part 1.2 - use two different HF QA models: use the default question-answering pipeline, then use other models of choice and discuss the differences in the result.

https://huggingface.co/docs/transformers/main_classes/pipelines

https://huggingface.co/docs/transformers/v4.35.0/en/main_classes/pipelines#transformers.QuestionAnsweringPipeline


In [13]:
!pip3 install -r ../requirements.txt

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from collections import defaultdict
import csv
import os

from src import utils
from src.question_answering import run, run_models


---
---

## Experiments & Results

For the first part, use the Hugging Face question-answering pipeline and feed it with the five 300-word long sections from the book of your choice that you analyzed in Project 1.

These sections should be selected so they are: **introducing the protagonist(s), the antagonist, the crime and crime scene, any significant evidence, and the resolution of the crime/a narrative that presents the case against the perpetrator.**

The questions you should ask are about the identity and characteristics of the protagonist, antagonist/perpetrator, the nature and the setting of the crime or crime scene, the evidence, and the case against the perpetrator.

Document the questions, ask the questions, and document the specificity and accuracy of the results.


In [15]:
# TODO: Try out a good selection of models and keep some interesting ones
models = [
    # DistilBERT
    "distilbert-base-cased-distilled-squad",
    "distilbert-base-uncased-distilled-squad",
    # RoBERTa
    "deepset/roberta-base-squad2",
    "deepset/roberta-large-squad2",
    # Deberta
    "deepset/deberta-v3-base-squad2",
    "deepset/deberta-v3-large-squad2",
    # Electra
    "deepset/electra-base-squad2",
]


In [16]:
ctxs = ["protagonist", "antagonist", "crime", "evidence", "resolution"]

csvfile = "res_qa.csv"

header = [
    "ctx_name",
    "ctx_fname",
    "question_idx",
    "question",
    "answer_true",
    "model_idx",
    "model_name",
    "answer_pred",
]

rows = []

for ctx_name in ctxs:
    for i, (fname, ctx) in enumerate(utils.read_context(ctx_name)):
        fname = os.path.basename(fname)
        print("#" * 80)
        print("#" * 80)
        print(ctx)

        # scores_by_question = {m: defaultdict(list) for m in models}
        scores_by_question = defaultdict(list)
        scores_by_answer = {m: defaultdict(list) for m in models}
        scores_by_model = defaultdict(list)

        for j, (question, true_answer) in enumerate(utils.read_qa(ctx_name)):
            print("=" * 80)
            print("=" * 80)
            print(f"Current Question: {question}")
            print(f"Expected Answer: {true_answer}")

            # for k in range(5):
            answers, scores = run_models(
                question,
                ctx,
                models,
                true_answer,
            )

            for k, (m, a, s) in enumerate(zip(models, answers, scores)):
                scores_by_model[m].append(s)
                scores_by_answer[m][true_answer].append(s)
                scores_by_question[m].append(
                    dict(
                        question=question,
                        answer=a,
                        expected_answer=true_answer,
                        score=s,
                    )
                )

                row = [
                    ctx_name,
                    fname,
                    j,
                    question,
                    true_answer,
                    k,
                    m,
                    a,
                ]

                for metric, score in s.items():
                    if metric not in header:
                        header.append(metric)
                    row.append(score)

                rows.append(row)

        # utils.create_plots(
        #     ctx_name, scores_by_model, scores_by_answer, scores_by_question
        # )

        # print()


with open(csvfile, "w") as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows)


Found: ['protagonist.0.md', 'protagonist.qa.md']
################################################################################
################################################################################
Sherlock Holmes took his bottle from the corner of the mantel-piece and his hypodermic syringe from its neat morocco case. With his long, white, nervous fingers he adjusted the delicate needle, and rolled back his left shirt-cuff. For some little time his eyes rested thoughtfully upon the sinewy forearm and wrist all dotted and scarred with innumerable puncture-marks. Finally he thrust the sharp point home, pressed down the tiny piston, and sank back into the velvet-lined arm-chair with a long sigh of satisfaction. Three times a day for many months I had witnessed this performance, but custom had not reconciled my mind to it. On the contrary, from day to day I had become more irritable at the sight, and my conscience swelled nightly within me at the thought that I had lacked the

Q: Who is the main character that the story revolves around?
A: Sherlock Holmes (model confidence score: 0.865)
--------------------------------------------------------------------------------
model: deepset/deberta-v3-base-squad2


Q: Who is the main character that the story revolves around?
A: Sherlock Holmes (model confidence score: 0.864)
--------------------------------------------------------------------------------
model: deepset/deberta-v3-large-squad2


Q: Who is the main character that the story revolves around?
A: Sherlock Holmes (model confidence score: 0.991)
--------------------------------------------------------------------------------
model: deepset/electra-base-squad2


Q: Who is the main character that the story revolves around?
A: It is cocaine (model confidence score: 0.0)
Current Question: Who is the detective in the story?
Expected Answer: Sherlock Holmes
--------------------------------------------------------------------------------
model: distilbert-base-cased-distilled-squad
Q: Who is the detective in the story?
A: Sherlock Holmes (model confidence score: 0.989)
--------------------------------------------------------------------------------
model: distilbert-base-uncased-distilled-squad
Q: Who is the detective in the story?
A: Sherlock Holmes (model confidence score: 0.866)
--------------------------------------------------------------------------------
model: deepset/roberta-base-squad2
Q: Who is the detective in the story?
A: Sherlock Holmes (model confidence score: 0.478)
--------------------------------------------------------------------------------
model: deepset/roberta-large-squad2
Q: Who is the detective in the story?
A: Sherlock Ho